In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
from PIL import ImageFilter, Image
import torchvision.models as models
import pretrainedmodels as ptmodels
import torch.nn as nn
import torch.nn.functional as F
import math
#import torch.utils.model_zoo as model_zoo
from torch.nn import init

from network.meso import Meso4, MesoInception4
from network.models import model_selection

#from ff import model_selection
from pipeline.metrics import accuracy_b, log_loss
from pipeline.model_methods import validate_img, train_img, validate_vid, train_vid
from pipeline.data_loaders import load_img_dataset, load_img_val_dataset, strong_aug
from pipeline.image_extracting import extract_faces, FastMTCNN

from pipeline.metrics import accuracy_sigmoid, accuracy_sigmoid_mean, log_loss_sigmoid, log_loss_b, accuracy_b_mean
from pipeline.model_methods import validate_vid_bf, train_vid_bf
from pipeline.image_extracting import extract_faces, FastMTCNN, InceptionResnetV1, extract_faces_dlib, MTCNN
from pipeline.blazeface import BlazeFace
from efficientnet_pytorch import EfficientNet

from pytorchcv.model_provider import get_model


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
PATH = "model.h5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
facedet = BlazeFace().to(device)
facedet.load_weights("pipeline/blazeface.pth")
facedet.load_anchors("pipeline/anchors.npy")
_ = facedet.train(False)

from pipeline.helpers.read_video_1 import VideoReader
from pipeline.helpers.face_extract_1 import FaceExtractor

frames_per_video = 17

video_reader = VideoReader()
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn, facedet)

In [3]:
y_train = pd.read_csv(r'D:\Machine Learning\deepfake-detection\data\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(100000, 19154)

In [4]:
#%%script false

batch_size = 8
normalize = torchvision.transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))

x_train = load_img_dataset(r'data\train', batch_size, resize=224, normalize=normalize)
x_val = load_img_val_dataset('data\img_val', 10, resize=224, normalize=normalize)

x_train_vid = r'D:\Machine Learning\deepfake-detection\data\train_set'
x_val_vid = r'data\val'
x_test_vid = r'D:\Machine Learning\deepfake-detection\data\cross_val'

In [5]:
class MyResNeXt(models.resnet.ResNet):
    def __init__(self, training=True):
        super(MyResNeXt, self).__init__(block=models.resnet.Bottleneck,
                                        layers=[3, 4, 6, 3], 
                                        groups=32, 
                                        width_per_group=4)
        #self.fc = nn.Linear(2048, 1)

In [6]:
model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=1)
#model = models.resnext50_32x4d(pretrained=True) 
#model =  models.resnext101_32x8d(pretrained=True) 
#model =  MyResNeXt()
#model = models.resnet18(pretrained=True)
#model._fc = nn.Linear(model._fc.in_features, 1)
#model.fc = nn.Linear(model.fc.in_features, 1)
'''
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(2048, 1)
)
'''

Loaded pretrained weights for efficientnet-b4


'\nmodel.fc = nn.Sequential(\n    nn.Dropout(0.5),\n    nn.Linear(2048, 1)\n)\n'

In [7]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [8]:
%%script false

#model.load_state_dict(torch.load("pretrained/resnext50_32x4d-7cdf4587.pth", map_location=device))
model.load_state_dict(torch.load("model.pth", map_location=device))
_ = model.eval()

Couldn't find program: 'false'


In [9]:
model = model.to(device)
loss =  torch.nn.BCELoss()

#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.) 
#optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.) 

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(x_train), epochs=10)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda epoch: epoch // 30, lambda epoch: 0.95 ** epoch], last_epoch=-1)

In [10]:
%%script false
found = False
for child in model.named_parameters():  
    if child[0] == 'layer4.0.conv2.weight':
        found = True
    child[1].requires_grad = found

Couldn't find program: 'false'


In [11]:
[i[0] for i in model.named_parameters() if i[1].requires_grad]

['_conv_stem.weight',
 '_bn0.weight',
 '_bn0.bias',
 '_blocks.0._depthwise_conv.weight',
 '_blocks.0._bn1.weight',
 '_blocks.0._bn1.bias',
 '_blocks.0._se_reduce.weight',
 '_blocks.0._se_reduce.bias',
 '_blocks.0._se_expand.weight',
 '_blocks.0._se_expand.bias',
 '_blocks.0._project_conv.weight',
 '_blocks.0._bn2.weight',
 '_blocks.0._bn2.bias',
 '_blocks.1._depthwise_conv.weight',
 '_blocks.1._bn1.weight',
 '_blocks.1._bn1.bias',
 '_blocks.1._se_reduce.weight',
 '_blocks.1._se_reduce.bias',
 '_blocks.1._se_expand.weight',
 '_blocks.1._se_expand.bias',
 '_blocks.1._project_conv.weight',
 '_blocks.1._bn2.weight',
 '_blocks.1._bn2.bias',
 '_blocks.2._expand_conv.weight',
 '_blocks.2._bn0.weight',
 '_blocks.2._bn0.bias',
 '_blocks.2._depthwise_conv.weight',
 '_blocks.2._bn1.weight',
 '_blocks.2._bn1.bias',
 '_blocks.2._se_reduce.weight',
 '_blocks.2._se_reduce.bias',
 '_blocks.2._se_expand.weight',
 '_blocks.2._se_expand.bias',
 '_blocks.2._project_conv.weight',
 '_blocks.2._bn2.weight',


In [12]:
%%script false

#0.37783308817654526 center crop
#0.3752234482172356  256
#0.40455125311689993 224
#0.44303366757688983 299

validate_img(model, x_val, y_train, loss, accuracy_b, device, 10, 
             print_results=True, 
             inference=False,
             checkpoint=None,
             reverse=False
            )

Couldn't find program: 'false'


In [13]:
0.37011126635215613 * 100 / 83, 0.37011126635215613 + 0.08228

(0.4459171883760917, 0.45239126635215615)

In [ ]:
#%%script false
# 38% / 47% / 57% /58.5% / 61% - 0.656% (0.99344%) - video
# 83% undersampling / 50% oversampling / 89.8% original dataset / 0.08228

metric, lost = train_img(model, loss, optimizer, scheduler, x_train, x_val, y_train, accuracy_b, device, [log_loss_sigmoid, accuracy_sigmoid_mean], face_extractor, 
                          epochs=10, 
                       batch_size=batch_size, 
                       del_net=False, 
                       useInference=False, 
                       inference=False, 
                       useScheduler=True,
                       checkpoint=[0.38, 0.99],
                         reverse=False
                      )


Validation: metrics  0.8579110384582294 loss  0.3383911053075265
Train: metrics  0.827476111723631 loss  0.374694469401107
Expected LB value: 0.4206711053075265 0.40770012687653795
Epoch: 1


In [ ]:
from operator import itemgetter
min_lost = max(enumerate(lost), key=itemgetter(1))[0] 
min_metrics = min(enumerate(metric), key=itemgetter(1))[0] 
print("Loss optim:", lost[min_lost], min_lost + 1)
print("Metrics optim:", metric[min_metrics], min_metrics + 1)

In [ ]:
plt.plot(metric, label='model')
plt.legend()
plt.title('Validation metrics')

In [ ]:
plt.plot(lost, label='model')
plt.legend()
plt.title('Validation Loss')

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
%%script false

# default - Validation: metrics  0.6033333333333334 loss:  0.6769222617149353

validate_vid_bf(model, x_val_vid, y_train, log_loss_sigmoid, accuracy_sigmoid_mean, device, 2, face_extractor, 
                fast_mtcnn=fast_mtcnn,
                print_results=True, 
                reverse=False)